In [ ]:
from setup_module import init_setup, import_data
from setup_module import import_data,save_and_load_list
import numpy as np
from classification_module import init_classification, combine_accuracy_graphs
import os
import matplotlib.pyplot as plt
import traceback
from sklearn.metrics import confusion_matrix
import seaborn as sns


In [ ]:
def init_setup():
    # check if there isnt a pre-prepared data
    if not os.path.exists("data"):
        # Create the folder if it doesn't exist
        os.makedirs("data")
    if not os.listdir("data"):
        filepath = "/home/bendegani/AMC_v2/GOLD_XYZ_OSC.0001_1024.hdf5"
        import_data(filepath)
    idx = 2
    print("loading data")
    # load the data
    samples = save_and_load_list(2, "cumulants_vec", idx)
    sample_dx = save_and_load_list(2, "cum_dx_vec", idx)
    sample_real = save_and_load_list(2, "cum_real_vec", idx)
    sample_imag = save_and_load_list(2, "cum_imag_vec", idx)
    labels = save_and_load_list(2, "labels", idx)
    snr = save_and_load_list(2, "snr_vec", idx)


    return samples, sample_dx, sample_real, sample_imag, labels, snr

def cumulant_fix_complex(cumulants):
    features_vec = []
    for ii in range(len(cumulants)):
        cum_abs = (abs(cumulants[ii]))
        # initiate output
        feats = []
        # for jj in range(len(cum_abs)):
        #     feats.append((cum_abs[jj]))
        feats.append((cum_abs[0])/cum_abs[1])
        feats.append(pow((cum_abs[1]),4)/cum_abs[7])
        feats.append(pow((cum_abs[2]),3)/pow(cum_abs[5],2))

        features_vec.append(feats)
    return features_vec

def cumulant_fix_iq(cum_real, cum_imag):
    # Gets the features vector from the cumulants
    features_vec = []
    for ii in range(len(cum_real)):
        real_abs = (abs(cum_real[ii]))
        imag_abs = (abs(cum_imag[ii]))
        # initiate output
        feats = []
        # for jj in range(len(real_abs)):
            # # feats.append((imag_abs[jj])/real_abs[jj]) 
            # feats.append((real_abs[jj]))
            # feats.append((imag_abs[jj]))
        feats.append((cum_real[0])/cum_imag[1])
        features_vec.append(feats)
    return features_vec

In [ ]:

samples, sample_dx, sample_real, sample_imag, labels, snr = init_setup()


In [ ]:
print("fixing for absolute value")
cumulants_vec = cumulant_fix_complex(samples)
cum_dx_vec = cumulant_fix_complex(sample_dx)
cum_iq_vec = cumulant_fix_iq(sample_real, sample_imag)
# convert to array
print("convert data")
labels = np.array(labels)
snr = np.array(snr)
samples = np.array(cumulants_vec)
samples_dx = np.array(cum_dx_vec)
sample_iq = np.array(cum_iq_vec)
sample_comb = np.concatenate((samples, samples_dx), 1)
    

In [ ]:

easy_mode = 1
group1 = [
        "OOK",
        "AM-SSB-WC",
        "AM-SSB-SC",
        "AM-DSB-WC",
        "AM-DSB-SC",
        "FM",
        "GMSK",
        "OQPSK",
    ]  # Low Order and Analog
group2 = [
        "4ASK",
        "8ASK",
        "16QAM",
        "32QAM",
        "64QAM",
        "128QAM",
        "256QAM",
    ]  # ASK and QAM
group3 = [
        "BPSK",
        "QPSK",
        "8PSK",
        "16PSK",
        "32PSK",
        "16APSK",
        "32APSK",
        "64APSK",
        "128APSK",
    ]  # PSK and APSK

if easy_mode == 1:
    easy_mods = [
        "OOK",
        "4ASK",
        "BPSK",
        "QPSK",
        "8PSK",
        "16QAM",
        "AM-SSB-SC",
        "AM-DSB-SC",
        "FM",
        "GMSK",
        "OQPSK",
    ]
    easy_mask = np.isin(labels, easy_mods)
    labels = labels[easy_mask]
    snr = snr[easy_mask]
    samples = samples[easy_mask]
    sample_comb = sample_comb[easy_mask]
    sample_iq = sample_iq[easy_mask]
    easy_mask = np.isin(labels, easy_mods)
    labels = labels[easy_mask]
    snr = snr[easy_mask]
    

    group1 = np.array(group1)[np.isin(group1,easy_mods)]
    group2 = np.array(group2)[np.isin(group2,easy_mods)]
    group3 = np.array(group3)[np.isin(group3,easy_mods)]
    
groups = [group1, group2, group3]

In [ ]:

print("we are here")
train_tresh = 10
print("classification for cumulants...")
accuracy_data10 = init_classification(samples, labels, snr, "cumulants", groups, easy_mode, train_tresh)
accuracy_data11 = init_classification(samples, labels, snr, "cumulants", groups, easy_mode, train_tresh-10)

print("classification for cumulants+dx...")
accuracy_data20 = init_classification(sample_comb, labels, snr, "cumulants_with_dx", groups, easy_mode, train_tresh)
accuracy_data21 = init_classification(sample_comb, labels, snr, "cumulants_with_dx", groups, easy_mode, train_tresh-10)
print("classification for cumulants IQ...")
accuracy_data30 = init_classification(sample_iq, labels, snr, "cumulants_iq", groups, easy_mode, train_tresh)
accuracy_data31 = init_classification(sample_iq, labels, snr, "cumulants_iq", groups, easy_mode, train_tresh-10)

combine_accuracy_graphs(accuracy_data10, accuracy_data20, accuracy_data30, train_tresh)
combine_accuracy_graphs(accuracy_data11, accuracy_data21, accuracy_data31, train_tresh-10)